In [1]:
# matplotlib inline plotting
%matplotlib inline
# make inline plotting higher resolution
%config InlineBackend.figure_format = 'svg'

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime
import statsmodels.api as sm

# Styling
plt.style.use('ggplot')

### Purpose
The purpose of this problem is to analyze the predictive ability of the Cyclically Adjusted
Price-Earnings (CAPE) ratio for future stock returns. The CAPE ratio is also known as the
price-smoothed-earnings ratio or as the Shiller $\mathrm{P} / \mathrm{E}$ ratio and the variable is available for free
download at Robert Shiller's website. The CAPE ratio is defined as the real stock price divided
by average real earnings over a ten-year period. It has been used in a series of articles by John
Campbell and Robert Shiller to examine long-horizon stock market predictability.

From our data-source we know that,
* ret: the return on the US stock market including all firms on the NYSE, AMEX, and NASDAQ 
* rf: proxy for the risk-free rate
* cape: cyclically adjusted price-earnings ratio

_I unfortunetly can't upload the data-file for this particular project._

In [2]:
# load and parse data
df = pd.read_excel('data/CAPE and US-market returns.xlsx', index_col=0, usecols='A:D')

df['parse_date'] = df.index
df['parse_date'] = df['parse_date'].apply(lambda x: datetime.strptime(str(x), '%Y%m'))

df = df.set_index('parse_date')
df.index.name = None

df.head()

,ret,rf,cape
1926-07-01,0.0318,0.0022,11.869694
1926-08-01,0.0289,0.0025,12.488808
1926-09-01,0.0059,0.0023,12.692615
1926-10-01,-0.0292,0.0032,12.426518
1926-11-01,0.0284,0.0031,12.615251


### Problem 1
Estimate long-horizon predictive regressions:
    
\begin{align*}
    r_{t \rightarrow t+k}=\alpha_{k}+\beta_{k} x_{t}+\varepsilon_{t \rightarrow t+k}\tag{1}
\end{align*}

where $r_{t \rightarrow t+k}$ is the log excess return on the US stock market from time $t$ to $t+k$ and $x_{t}$ is the log CAPE ratio at time $t$. Consider horizons in the range from one month up to ten years: $k=1,6,12,24,36,48,60,72,84,96,108,$ and $120 .$ Report and compare the $\beta_{k}$ coefficients and $R^{2}$ statistics across the forecast horizons. The sample period is from **1926:m7** to **2020:m7**

In [3]:
# settings
windows = [1, 6, 12, 24, 36, 48, 60, 72, 84, 96, 108, 120]
n_lags = 1

Lets first check if we have selected the right data...

In [4]:
print(f'First observation: {df.iloc[0].name}')
print(f'Last observation: {df.iloc[-1].name}')

First observation: 1926-07-01 00:00:00
Last observation: 2020-07-01 00:00:00


Now let's start out by creating log excess return $t\rightarrow t+k$ and $x_t$ as the log CAPE ratio at time $t$

In [5]:
# t net returns/rate
df['ret'] = df['ret'] + 1
df['rf'] = df['rf'] + 1

# log t CAPE
df['lcape'] = np.log(df['cape'])
df.head()

,ret,rf,cape,lcape
1926-07-01,1.0318,1.0022,11.869694,2.473988
1926-08-01,1.0289,1.0025,12.488808,2.524833
1926-09-01,1.0059,1.0023,12.692615,2.541020
1926-10-01,0.9708,1.0032,12.426518,2.519833
1926-11-01,1.0284,1.0031,12.615251,2.534906


To calculate $t\rightarrow t+k$ period log excess return we need to write a small function, which loops the series for both risk-free rate and stock-returns taking the product for each k-period window. 

We then log these values and take the difference with the risk-free rate to have our $t\rightarrow t+k$ period changes.

In [6]:
def rolling_backwards_product(s, k):
    # reverse series to make t->(t+k) change
    s = s.sort_index(ascending=False)
    # make rolling series
    s = s.rolling(k)
    
    return s.apply(np.prod, raw=True)


for window in windows:
    df[f'{window}_leret'] = np.log(rolling_backwards_product(df['ret'], window)) - np.log(rolling_backwards_product(df['rf'], window))


display(df.head())

,ret,rf,cape,lcape,1_leret,6_leret,12_leret,24_leret,36_leret,48_leret,60_leret,72_leret,84_leret,96_leret,108_leret,120_leret
1926-07-01,1.0318,1.0022,11.869694,2.473988,0.029107,0.076557,0.151879,0.395039,0.695591,0.348264,0.031850,-1.055330,-0.112727,-0.160007,-0.070980,0.304169
1926-08-01,1.0289,1.0025,12.488808,2.524833,0.025993,0.046851,0.192655,0.372093,0.709978,0.359452,-0.065708,-0.793039,-0.243071,-0.305186,-0.027681,0.339625
1926-09-01,1.0059,1.0023,12.692615,2.541020,0.003585,0.061703,0.186116,0.410563,0.762309,0.336451,-0.087611,-0.503864,-0.155322,-0.276886,-0.027522,0.323481
1926-10-01,0.9708,1.0032,12.426518,2.519833,-0.032830,0.059413,0.228937,0.435296,0.702673,0.196794,-0.435396,-0.537282,-0.271492,-0.282774,-0.005150,0.329647
1926-11-01,1.0284,1.0031,12.615251,2.534906,0.024909,0.096821,0.217823,0.481284,0.512011,0.137814,-0.325310,-0.645640,-0.325956,-0.266682,0.095612,0.431243


Now we can simply estimate the long-run predictive regression from $(1)$ and report our estimates - we'll use the statsmodels package here.

In [7]:
def slice_dataframe(k):
    # slice dataframe for relevant information
    reg = df[[f'{k}_leret','lcape']].copy()

    # when using predictive regressions we lag our return vector
    reg['lcape'] = reg['lcape'].shift(n_lags)

    # drop all nan's
    reg.dropna(inplace=True)
    
    return (reg['lcape'], reg[f'{k}_leret'])


# fit model using statsmodels - should be self explainatory
def fit_model(k):
    X, y = slice_dataframe(k)
    X = sm.add_constant(X)

    res = sm.OLS(y, X).fit()
    
    return {'windows': window,
            'beta_k': res.params[1],
            'R squared': res.rsquared}


# save and make output pretty
results = pd.DataFrame(columns=['windows', 'beta_k', 'R squared'])

for window in windows:
    results = results.append(fit_model(window), ignore_index=True)

results

,windows,beta_k,R squared
0,1.0,-0.009782,0.005432
1,6.0,-0.061759,0.032798
2,12.0,-0.136438,0.072695
3,24.0,-0.263625,0.133141
4,36.0,-0.353641,0.169402
5,48.0,-0.419058,0.189981
6,60.0,-0.460648,0.198330
7,72.0,-0.475547,0.209949
8,84.0,-0.516682,0.239954
9,96.0,-0.583363,0.273459


### Solution
We see that $R^2$ increase with $k$ - thus the predictive power of CAPE increase with the time-horizon. We expect that the price-earnings ratio is highly persistent and slow to mean-revert due to high $\rho$ in our MA-process for the error-term $\varepsilon_{t \rightarrow t+k} \sim M A(k-1)$. 

Intuitively this means that when prices are high relative to earnings, we'd expect lower stock returns.